In [ ]:
import torch
import lib.utils.bookkeeping as bookkeeping
from torch.utils.data import DataLoader
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import lib.models.models as models
import lib.models.model_utils as model_utils
from lib.datasets import mnist, maze, protein, synthetic
import lib.datasets.dataset_utils as dataset_utils
import lib.losses.losses as losses
import lib.losses.losses_utils as losses_utils
import lib.training.training as training
import lib.training.training_utils as training_utils
import lib.optimizers.optimizers as optimizers
import lib.optimizers.optimizers_utils as optimizers_utils
import lib.sampling.sampling as sampling
import lib.sampling.sampling_utils as sampling_utils
import os
from tqdm import tqdm
from lib.datasets.maze import maze_acc
from ruamel.yaml.scalarfloat import ScalarFloat
from lib.datasets.mnist_fid import evaluate_fid_score
def categorical_log_likelihood(x, logits):
    """Log likelihood of a discretized Gaussian specialized for image data.

    Assumes data `x` consists of integers [0, num_classes-1].

    Args:
    x: where to evaluate the distribution. shape = (bs, ...), dtype=int32/int64
    logits: logits, shape = (bs, ..., num_classes)

    Returns:
    log likelihoods
    """
    pass
    #log_probs = nn.log_softmax(logits)
    #x_onehot = nn.one_hot(x, logits.shape[-1])
    #return torch.sum(log_probs * x_onehot, axis=-1)

In [ ]:
max_t = 0.99999
min_t = 0.01
num_steps = [342]
nfe_10_01 = []
for steps in num_steps:
    ts = np.concatenate(
        (np.
        linspace(max_t, min_t, steps), np.array([0]))
    )

    k = 0 
    for i, t in (enumerate(ts[0:-1])):
        #print(i+1)
        if t <= 0.1:
            for j in range(5):
                k += 1
                #print(k)

    nfe_10_01.append(i+1+k)
print(nfe_10_01)

# nfe_1_01 = [54, 109, 272, 545, 818, 1090] 
# nfe_3_01 = [64, 129, 318, 637, 956, 1272]
# nfe_5_01 = [74, 149, 364, 729, 1094, 1454]
# nfe_10_01 = [99, 199, 479, 959, 1439, 1909]

# pc_1_steps = 1 [4, 9, 18, 27, 45, 91] => NFE [5, 10, 20, 30, 50, 100]
# pc_3_steps = [7, 14, 23, 38, 78] => NFE [10, 20, 32, 50, 102]
# pc_5_steps = [10, 20, 33, 67] => NFE [15, 30, 48, 102]

# 1 [229, 458] => 250, 500
# 3 [196, 392]  => 500
# 5 [170, 342]

In [ ]:

n_samples = 15 * 25
#s_name = 'Samples/mnist_unet_LBJF2000.npy'
s_name = 'Samples/MNIST/mnist_unet_corrector5_01_TauL1000.npy' # Samples/MNIST/
samples = np.load(s_name)
samples = samples[550:]
samples = samples.reshape(-1, 1, 28, 28)
#saving_train_path = os.path.join(cfg.saving.sample_plot_path, f"{cfg.model.name}{state['n_iter']}_{cfg.sampler.name}{cfg.sampler.num_steps}.png")
fig = plt.figure(figsize=(15, 25)) 
plt.subplots_adjust(wspace=0.001, hspace=0.001)
int(np.sqrt(n_samples))
for i in range(n_samples):
    plt.subplot(int(25), int(15), 1 + i)
    plt.axis("off")
    plt.imshow(np.transpose(samples[i, ...], (1,2,0)), cmap="gray")

plt.tight_layout()  
plt.savefig('mnist_unet_corrector5_01_TauL1000.pdf', transparent=True)
plt.show()
plt.close()

In [ ]:

path = 'SavedModels/MNISTHollow/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2024-01-18' # 2
config_name = 'config_001_hollow_direct.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_519999_hollow_direct.pt'

path = 'SavedModels/MNISTHollow/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2024-01-03' # 2
config_name = 'config_001_hollow.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_539999_hollow.pt'

path = 'SavedModels/MNIST/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
date = '2023-12-25' # 2
config_name = 'config_001_unet14Mlogits.yaml' # config_001_hollowMLEProb.yaml
model_name = 'model_599999_unet14Mlogits.pt'

dataset_location = 'lib/datasets'

config_path = os.path.join(path, date, config_name)
checkpoint_path = os.path.join(path, date, model_name)

cfg = bookkeeping.load_config(config_path)
cfg.data.train = False
"""
if cfg.loss.name == 'CatRM':
    cfg.loss.name = 'SDDMNLL'
elif cfg.loss.name == 'CTElbo':
    cfg.loss.name = 'TauLDRNLL'
else:
    raise KeyError("Eval not defined!")

cfg.training.train_step_name = 'EvalMNIST'
"""
device = torch.device(cfg.device)

model = model_utils.create_model(cfg, device)

optimizer = optimizers_utils.get_optimizer(model.parameters(), cfg)

state = {"model": model, "optimizer": optimizer, "n_iter": 0}

state = bookkeeping.load_state(state, checkpoint_path)
cfg.training.n_iters = 600000 
cfg.sampler.sample_freq = 100000000
cfg.saving.checkpoint_freq = 10000
cfg.sampler.num_steps = 1000

# EvalMNIST

loss = losses_utils.get_loss(cfg)

training_step = training_utils.get_train_step(cfg)

#sampler = sampling_utils.get_sampler(cfg)



print("Info:")
print("--------------------------------")
print("State Iter:", state["n_iter"])
print("--------------------------------")
print("Name Dataset:", cfg.data.name)
print("Loss Name:", cfg.loss.name)

print("--------------------------------")
print("Model Name:", cfg.model.name)
print("Number of Parameters: ", sum([p.numel() for p in model.parameters()]))

model.eval()
#66.52994572680203  # Inception mean:  7.715643802044586
# Inception std:  0.18737017997662175

In [ ]:
n_rounds = 150
nll = []
dataset = dataset_utils.get_dataset(cfg, device, dataset_location)

dataloader = torch.utils.data.DataLoader(dataset,
    batch_size=cfg.data.batch_size,
    shuffle=cfg.data.shuffle)

for i, minibatch in tqdm(enumerate(tqdm(dataloader))):
    minibatch = minibatch.to(device)
    l = training_step.step(state, minibatch.long(), loss)
    nll.append(l.item())

    if i >= n_rounds:
        print("Avg NLL:", np.mean(nll))
        break

In [ ]:
steps = [1000]
#mjumps_rate = []
#chang_dim_all = np.empty((5, 1000))
#mjumps_all = np.empty((5, 1000))
avg_dim_1 = []
avg_dim_12 = []
avg_dim_ins = []
avg_dim_mulj = []
chang_dim = []
cfg.sampler.num_corrector_steps = 0
cfg.sampler.corrector_entry_time = ScalarFloat(0.0)
#cfg.sampler.num_steps = 10
cfg.sampler.is_ordinal = True
sampler_names = ['MidPointTauL'] # 'CRMLBJF', 
n_samples = 200
clip1 = []
clip2 = []
for _, n_name in enumerate(sampler_names):
    cfg.sampler.name = n_name 
    print("# name:", cfg.sampler.name)
    for _, n_steps in enumerate(steps):
        cfg.sampler.num_steps = n_steps
        print("# step:", cfg.sampler.num_steps)
        for i in range(50):
            
            sampler = sampling_utils.get_sampler(cfg)
            samples, changes_jump, changes_dim, change_dim_first, change_1to2, change_clamp1, change_clamp2 = sampler.sample(model, n_samples)
            saved_samples = samples
            print(f'mnist_hollow_{cfg.sampler.name}{cfg.sampler.num_steps}_mnist{i}')
            np.save(f'mnist_hollow_{cfg.sampler.name}{cfg.sampler.num_steps}_mnist{i}', saved_samples.reshape(n_samples, 1, cfg.data.image_size, cfg.data.image_size))
        """
            if i == 0 :
                print("# Prop Multiple jumps over time", changes_jump)
                print("# change dim x to x_prime over time", change_dim_first)
                print("# change dim x_prime to x_new over time", change_1to2)
                print("# Average proportion of dim that changes over time", changes_dim)
                print("-----------------------------------------------")
                print("# Mean of multiple jumps for tau plot", np.mean(changes_jump))
                print("# FIRST: Mean of avg changing dim for tau plot", np.mean(change_dim_first))
                print("# FIRST2SECOND: Mean of avg changing dim for tau plot", np.mean(change_1to2))
                print("# Mean of avg changing dim for tau plot", np.mean(changes_dim))

            
        
        #np.save(f'MNISTstatistics/unet_mptaul{n_steps}_mjumps_prime_t.npy', changes_jump)
        #np.save(f'MNISTstatistics/unet_mptaul{n_steps}_dim_1_prime_t.npy', change_dim_first)
        #np.save(f'MNISTstatistics/unet_mptaul{n_steps}_dim_12_prime_t.npy', change_1to2)
        #np.save(f'MNISTstatistics/unet_mptaul{n_steps}_dim_ins_prime_t.npy', changes_dim)
        
        avg_dim_1.append(np.mean(change_dim_first))
        avg_dim_12.append(np.mean(change_1to2))
        avg_dim_ins.append(np.mean(changes_dim))
        avg_dim_mulj.append(np.mean(changes_jump))
        clip1.append(np.mean(change_clamp1))
        clip2.append(np.mean(change_clamp2))
        """
#np.save(f'MNISTstatistics/unet_mptaul_mjumps_tau.npy', avg_dim_mulj)
#np.save(f'MNISTstatistics/unet_mptaul_dim_1_tau.npy', avg_dim_1)
#np.save(f'MNISTstatistics/unet_mptaul_dim_12_tau.npy', avg_dim_12)
#np.save(f'MNISTstatistics/unet_mptaul_dim_ins_tau.npy', avg_dim_ins)
#np.save(f'MNISTstatistics/unet_mptaul_clip1.npy', clip1)
#np.save(f'MNISTstatistics/unet_mptaul_clip2.npy', clip2)

In [ ]:
steps = [5, 7, 10, 15, 20, 30, 50, 100, 250, 500, 1000]
avg_dim_1 = []
avg_dim_12 = []
avg_dim_ins = []
avg_dim_mulj = []
for n_steps in steps:
    changes_jump = np.load(f'MNISTstatistics/unet_mptaul{n_steps}_mjumps_prime_t.npy')
    change_dim_first = np.load(f'MNISTstatistics/unet_mptaul{n_steps}_dim_1_prime_t.npy')
    change_1to2 = np.load(f'MNISTstatistics/unet_mptaul{n_steps}_dim_12_prime_t.npy')
    changes_dim = np.load(f'MNISTstatistics/unet_mptaul{n_steps}_dim_ins_prime_t.npy')
    avg_dim_1.append(np.mean(change_dim_first))
    avg_dim_12.append(np.mean(change_1to2))
    avg_dim_ins.append(np.mean(changes_dim))
    avg_dim_mulj.append(np.mean(changes_jump))
np.save(f'MNISTstatistics/unet_mptaul_mjumps_prime_tau.npy', avg_dim_mulj)
np.save(f'MNISTstatistics/unet_mptaul_dim_1_prime_tau.npy', avg_dim_1)
np.save(f'MNISTstatistics/unet_mptaul_dim_12_prime_tau.npy', avg_dim_12)
np.save(f'MNISTstatistics/unet_mptaul_dim_ins_prime_tau.npy', avg_dim_ins)
    

    

In [ ]:
# pc_1_steps = 1 [4, 9, 18, 27, 45, 91] => NFE [5, 10, 20, 30, 50, 100]
# pc_3_steps = [7, 14, 23, 38, 78] => NFE [10, 20, 32, 50, 102]
# pc_5_steps = [10, 20, 33, 67] => NFE [15, 30, 48, 102]
# 1 [229, 458] => 250, 500
# 3 [196, 392]  => 500
# 5 [170, 342]

cfg.sampler.num_corrector_steps = 0
cfg.sampler.corrector_entry_time = ScalarFloat(0.1)
#cfg.sampler.num_steps = 10
cfg.sampler.is_ordinal = True

sampler_names = ['TauL'] # 'CRMLBJF', 
n_samples = 500
for _, n_name in enumerate(sampler_names):
    cfg.sampler.name = n_name 
    print("# name:", cfg.sampler.name)
    for pc in [3, 5]:
        cfg.sampler.num_corrector_steps = pc
        print("Corrector:", cfg.sampler.num_corrector_steps)
        if cfg.sampler.num_corrector_steps == 3:
            steps = [57, 392] # 75, 500
        elif cfg.sampler.num_corrector_steps == 5:
            steps = [50, 170, 342] # 75, 250, 500, 
        else:
            raise ValueError("Not existing!")
        for _, n_steps in enumerate(steps):
            cfg.sampler.num_steps = n_steps
            print("# step:", cfg.sampler.num_steps)
            if n_steps == 392:
                numbers = 11
            else:
                numbers = 20
            for i in range(numbers):
                sampler = sampling_utils.get_sampler(cfg)
                samples, changes = sampler.sample(model, n_samples)
                saved_samples = samples
                np.save(f'mnist_unet_corrector{pc}_01_{cfg.sampler.name}{cfg.sampler.num_steps}_mnist{i}', saved_samples.reshape(n_samples, 1, cfg.data.image_size, cfg.data.image_size))


In [ ]:

c = 5
steps = [342]
sampler_name = ['TauL']# , CRMTauL']
for sampler_n_i in sampler_name:
    for step in steps:
        data_list = []
        for i in range(20):
            print(f'mnist_unet_corrector{c}_01_{sampler_n_i}{step}_mnist{i}.npy')
            data_list.append(np.load(f'mnist_unet_corrector{c}_01_{sampler_n_i}{step}_mnist{i}.npy'))
        data = np.vstack(data_list)
        print(data.shape)
        np.save(f'Samples/MNIST/mnist_unet_corrector{c}_01_{sampler_n_i}{step}.npy', data)

In [ ]:
steps = [57] 
sampler_names = ['TauL']# , 'CRMTauL', 'ExactSampling' ] # 'LBJF', 
c = 3
#for sampler_n_i in sampler_name:
#    for step in steps:
cfg.data.train = False
#        data = np.load(f'unet_{sampler_n_i}{step}_mnist.npy')
fid_values = []
for sampler_n in sampler_names:
    for step in steps:
        data = np.load(f'Samples/MNIST/mnist_unet_corrector{c}_01_{sampler_n}{step}.npy')
        #data = np.load('mpl_x.npy')
        print(data.shape)
        dataset = dataset_utils.get_dataset(cfg, device, dataset_location)

        dataloader = torch.utils.data.DataLoader(dataset,
            batch_size=data.shape[0],
            shuffle=cfg.data.shuffle)
        for true_data in tqdm(dataloader):
            #print(f'mnist_hollow_{sampler_n}{step}.npy')
            print("----------------------------------")
            print("Sampler:", sampler_n)
            print("Step:", step)
            fid = evaluate_fid_score(data, true_data.cpu().numpy(), 100)
            print("FID:", fid)
            fid_values.append(fid)
            break
print(fid_values)
# LBJF: 500 c1
# LBJF ab 500 c3: [65.00767287991044, 30.49072227318959, 17.539240669487356]
# LBJF ab 500 c5: [62.703384997916515, 28.830249202615704, 17.792121677059924]
# pc_1_steps = 1 [4, 9, 18, 27, 45, 91] => NFE [5, 10, 20, 30, 50, 100]
# # nfe_01_1 = [355.30401756086303, 299.8788404145761, 220.94562240586495, 63.526149427266375, 17.06536557569376, 2.813092226544086]
# pc_3_steps = [7, 14, 23, 38, 78] => NFE [10, 20, 32, 50, 102]
# nfe_01_3 = [368.2020780292752, 306.44852994200113, 105.99785122999572, 23.8096318610784, 3.8135712497214342]
# pc_5_steps = [10, 20, 33, 67] => NFE [15, 30, 48, 102]
# nfe_01_5 = [308.4621460802384, 153.52888664861723, 32.976239962089096, 5.501552200792844]

# 0: 75:  4.3245049017693304
# 1: 250, 500: [5.845140920974131, 2.252598541276768, 2.3627632134412977]
# 3: 250, 500, [9.18351245256298 2.3423048205916643, [[2.3001590957738642]]
# 5: 75, 250, 500, [12.95, 2.3731947632869037, 2.418666817264665]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
marker_size = 4
line_width = 1.5
# FID
steps_plot = np.round(1 / np.array([5, 10, 20, 50, 100, 250, 500, 1000, 2000]), 4)
steps = 1 / np.array([5, 7, 10, 15, 20, 30, 50, 100, 250, 500, 750, 1000, 1500])
fid_taul = np.array([369.82769160402194, 339.7123739152373, 283.87526811768316, 212.90508021598973, 120.04226304249106, 40.6234648618652, 13.453524786963495, 3.6, 2.98404509903506, 2.5895556730263706, 2.51, 2.48, 2.460157370950734])
fid_lbjf = np.array([369.82769160402194, 368.6921461974128, 367.43394813969206, 363.92216793023744, 360.795573791034, 354.6901690476625, 339.59518918406286, 305.2288604477556, 212.0276736955564, 75.84530505097865, 32.956622302615955, 19.817409823054646, 7.19])
fid_mpl = np.array([344.39284875166595, 290.45552043389864, 247.32958569058357, 88.0448768470724, 42.82853224434638, 12.394969584434108, 4.020041973904654, 3.3680633408595213, 2.8853011388733307, 2.5683849558007807, 2.5, 2.4493237174940248, 2.40])
plt.plot(steps, fid_taul, marker='o', color='green', label='Tau-Leaping', markersize=marker_size, linewidth=line_width)
plt.plot(steps, fid_lbjf, marker='o', label='Euler', markersize=marker_size, linewidth=line_width)
plt.plot(steps, fid_mpl, marker='o', label='Midpoint Tau-Leaping', color='lightcoral', markersize=marker_size, linewidth=line_width)
plt.xlabel('$\\tau$', fontsize=13)
plt.ylabel('FID Value', fontsize=13)
plt.title('FID Value of Sampling Methods for Varying $\\tau$', fontsize=14)
plt.legend(fontsize=9)
plt.grid(True)
plt.xscale('log')
steps_y = [370, 250, 200, 150, 100, 50, 25, 10, 7, 4, 2.5, 2]
plt.yscale('log')
plt.yticks(steps_y, steps_y)
plt.xticks(steps_plot, steps_plot)
plt.savefig('fid_tau.pdf')
plt.show()

In [ ]:
# IS 
import numpy as np
import matplotlib.pyplot as plt
marker_size = 4
line_width = 1.5
steps_plot = np.round(1 / np.array([5, 10, 20, 50, 100, 250, 500, 1000, 2000]), 4)
mean_is_mpl = np.array([8.56, 8.62, 8.677, 8.7, 8.733, 8.763])
std_is_mpl = np.array([0.078, 0.1, 0.079, 0.06, 0.057, 0.075])
# 30-60, 50-100, 100-200, 250-500, 500-1000, 750, 1500

steps = 1 / np.array([5, 7, 10, 15, 20, 30, 50, 100, 250, 500, 750, 1000, 1500])
print(len(steps))
mean_is_taul = np.array([2.19, 2.22, 2.708047858197907, 3.86, 4.946, 6.260149524787618, 7.725075911310337, 8.600311295819452, 8.695352062102907, 8.730387442552915, 8.76098102664333, 8.785004667417176, 8.789941441541984])
std_is_taul = np.array([0.029, 0.027, 0.035763717879784655, 0.07, 0.09223751449123928, 0.05542292679137494, 0.07471893518876059, 0.11749863261977536, 0.05961257597104094, 0.057613531976108634, 0.06736065870254852, 0.07408158674264428, 0.07343111567356189])

mean_is_lbjf = np.array([2.197, 2.1935, 2.1905016592910244, 2.21, 2.233561128867508, 2.2720786975418426, 2.3474579482515194, 2.703195286772994, 4.422914663421897, 6.212356745289218, 7.003172870037281, 7.3776977749287855, 8.086100926383237])
std_is_lbjf = np.array([0.017, 0.028, 0.016885548328526935, 0.023, 0.01655479275116797, 0.01665546477191724, 0.021260056132834024, 0.042065063979126655, 0.0653358704449712, 0.08250910888402853, 0.09807975265868278, 0.11617335996188316, 0.07638698691732632])

mean_is_mpl = np.array([2.44, 3.01, 3.92, 5.575, 6.35, 7.779, 8.56, 8.65, 8.7, 8.745, 8.763, 8.795, 8.8])
std_is_mpl = np.array([0.019, 0.0633, 0.046, 0.0665, 0.0776, 0.087, 0.078, 0.1, 0.079, 0.06, 0.075, 0.082, 0.072])
print(len(mean_is_mpl), len(mean_is_taul), len(mean_is_lbjf), len(std_is_mpl), len(std_is_taul), len(std_is_lbjf))

plt.plot(steps, mean_is_taul, marker='o', color='green', label='Tau-Leaping', markersize=marker_size, linewidth=line_width)
plt.fill_between(steps, [mean - std for mean, std in zip(mean_is_taul, std_is_taul)],
                 [mean + std for mean, std in zip(mean_is_taul, std_is_taul)],
                 alpha=0.3, color='green')
plt.plot(steps, mean_is_lbjf, marker='o', label='Euler', markersize=marker_size, linewidth=line_width)
plt.fill_between(steps, [mean - std for mean, std in zip(mean_is_lbjf, std_is_lbjf)],
                 [mean + std for mean, std in zip(mean_is_lbjf, std_is_lbjf)],
                 alpha=0.3)

plt.plot(steps, mean_is_mpl, marker='o', label='MidPoint Tau-Leaping', color='lightcoral', markersize=marker_size, linewidth=line_width)
plt.fill_between(steps, [mean - std for mean, std in zip(mean_is_mpl, std_is_mpl)],
                 [mean + std for mean, std in zip(mean_is_mpl, std_is_mpl)],
                 alpha=0.3)

plt.xlabel('$\\tau$', fontsize=13)
plt.ylabel('Inception Score', fontsize=13)
plt.title('Inception Score of Sampling Methods for Varying $\\tau$', fontsize=14)
plt.legend()
plt.grid(True)
plt.xscale('log')
#plt.yscale('log')
plt.xticks(steps_plot, steps_plot)
plt.savefig('inception_score_tau.pdf')
plt.show()



In [ ]:
# NFE FID

import numpy as np
import matplotlib.pyplot as plt
marker_size = 4
line_width = 1.5

nfe_mpl = [100, 200, 500, 1000, 1250, 1500]
nfe_taul = [100, 250, 500, 750, 1000, 1250, 1500]

# steps = 1 / np.array([5, 7, 10, 15, 20, 30, 50, 100, 250, 500, 750, 1000, 2000])
nfe_mpl = [100, 200, 500, 1000, 1250, 1500]
nfe_taul = [100, 250, 500, 750, 1000, 1250, 1500]
fid_taul = np.array([3.6, 2.98404509903506, 2.5895556730263706, 2.51, 2.48, 2.47, 2.460157370950734])
fid_mpl = np.array([4.020041973904654, 3.3680633408595213, 2.8853011388733307, 2.5683849558007807, 2.52, 2.5])
steps_y = [4, 3.6, 3.35, 3, 2.88, 2.56, 2.51, 2.46]
steps_plot = [100, 250, 500, 750, 1000, 1250, 1500]

nfe_taul_5100 = [10, 20, 30, 50, 100]# , 250, 500, 750, 1000, 1500], 2.98404509903506, 2.5395556730263706, 2.51, 2.48, 2.460157370950734])
nfe_mpl_5100 = [10, 20, 30, 40, 60, 100]#, 200, 500, 1000, 1500]
fid_taul_5100 = np.array([283.87526811768316, 120.04226304249106, 40.6234648618652, 13.453524786963495, 3.6]) # 2.98404509903506, 2.5895556730263706, 2.51, 2.48, 2.460157370950734])
fid_mpl_5100 = np.array([344.39284875166595, 247.32958569058357, 88.0448768470724, 42.82853224434638, 12.394969584434108, 4.020041973904654]) #3.3680633408595213, 2.8853011388733307, 2.5683849558007807, 2.5])
steps_y_5100 = [345, 250, 125, 50, 30, 15, 10, 3.5]
steps_plot_5100 = [10, 15, 20, 30, 50, 100]

#steps_plot = 
#plt.figure(figsize=(10, 6))
plt.plot(nfe_taul, fid_taul, marker='o', color='green', label='Tau-Leaping', markersize=marker_size, linewidth=line_width)
plt.plot(nfe_mpl, fid_mpl, marker='o', label='Midpoint Tau-Leaping', color='lightcoral', markersize=marker_size, linewidth=line_width)
#plt.plot(nfe_1_01, nfe_1_01_fid, marker='o', label='Tau-Leaping C1')
#plt.plot(nfe_3_01, nfe_3_01_fid, marker='o', label='Tau-Leaping C3')
#plt.plot(nfe_5_01, nfe_5_01_fid, marker='o', label='Tau-Leaping C5')
#plt.plot(nfe_10_01, nfe_10_01_fid, marker='o', label='Tau-Leaping C10')
plt.xlabel('NFE', fontsize=13)
plt.ylabel('FID Value', fontsize=13)
plt.title('FID Value of Tau-Leaping Methods for Varying NFE', fontsize=14)
plt.legend()
plt.grid(True)
plt.yscale('log')
plt.yticks(steps_y, steps_y)
plt.xticks(steps_plot, steps_plot)
#
plt.savefig('fid_nfe_mp_taul.pdf')
plt.show()

#steps_plot = 
#plt.figure(figsize=(10, 6))
plt.plot(nfe_taul_5100, fid_taul_5100, marker='o', color='green', label='Tau-Leaping', markersize=marker_size, linewidth=line_width)
plt.plot(nfe_mpl_5100, fid_mpl_5100, marker='o', label='Midpoint Tau-Leaping', color='lightcoral', markersize=marker_size, linewidth=line_width)
#plt.plot(nfe_1_01, nfe_1_01_fid, marker='o', label='Tau-Leaping C1')
#plt.plot(nfe_3_01, nfe_3_01_fid, marker='o', label='Tau-Leaping C3')
#plt.plot(nfe_5_01, nfe_5_01_fid, marker='o', label='Tau-Leaping C5')
#plt.plot(nfe_10_01, nfe_10_01_fid, marker='o', label='Tau-Leaping C10')
plt.xlabel('NFE', fontsize=13)
plt.ylabel('FID Value', fontsize=13)
plt.title('FID Value of Tau-Leaping Methods for Varying NFE', fontsize=14)
plt.legend()
plt.grid(True)
plt.yscale('log')
plt.yticks(steps_y_5100, steps_y_5100)
plt.xticks(steps_plot_5100, steps_plot_5100)
#
plt.savefig('fid_nfe_mp_taul.pdf')
plt.show()


In [ ]:
# NFE IS
marker_size = 4
line_width = 1.5
mean_is_taul = np.array([8.600311295819452, 8.695352062102907, 8.730387442552915, 8.76098102664333, 8.785004667417176, 8.787, 8.789941441541984])
std_is_taul = np.array([0.11749863261977536, 0.05961257597104094, 0.057613531976108634, 0.06736065870254852, 0.07408158674264428, 0.056, 0.07343111567356189])

mean_is_mpl = np.array([8.56, 8.65, 8.7, 8.745, 8.753, 8.763])
std_is_mpl = np.array([0.078, 0.1, 0.079, 0.06, 0.057, 0.075])
#50-100, 100-200, 250-500, 500-1000, 750, 1500

steps = 1 / np.array([5, 7, 10, 15, 20, 30, 50, 100, 250, 500, 750, 1000, 1500])

nfe_mpl = [100, 200, 500, 1000, 1250, 1500]
nfe_taul = [100, 250, 500, 750, 1000, 1250, 1500]

plt.plot(nfe_taul, mean_is_taul, marker='o', color='green', label='Tau-Leaping', markersize=marker_size, linewidth=line_width)
#plt.fill_between(nfe_taul, [mean - std for mean, std in zip(mean_is_taul, std_is_taul)],[mean + std for mean, std in zip(mean_is_taul, std_is_taul)],alpha=0.3, color='green')

plt.plot(nfe_mpl, mean_is_mpl, marker='o', label='MidPoint Tau-Leaping', color='lightcoral', markersize=marker_size, linewidth=line_width)
#plt.fill_between(nfe_mpl, [mean - std for mean, std in zip(mean_is_mpl, std_is_mpl)],[mean + std for mean, std in zip(mean_is_mpl, std_is_mpl)],alpha=0.3)

plt.xlabel('NFE', fontsize=13)
plt.ylabel('Inception Score', fontsize=13)
plt.title('Inception Score of Tau-Leaping Methods for Varying NFE', fontsize=14)
plt.legend()
plt.grid(True)
steps_plot = [100, 250, 500, 750, 1000, 1250, 1500]
steps_y = [8.56, 8.6,8.65, 8.7, 8.74, 8.76, 8.79]
#plt.yscale('log')
plt.xticks(steps_plot, steps_plot)
plt.yticks(steps_y, steps_y)
plt.savefig('inception_score_mp_tau.pdf')
plt.show()

nfe_1_01 = [54, 109, 272, 545, 818, 1000] # 1090 last
nfe_3_01 = [64, 129, 318, 637, 1000, 1272] # vorletzes 956
nfe_5_01 = [74, 149, 364, 729, 1000, 1500] #vorletztes 1094 letztes 15000
nfe_10_01 = [99, 199, 479, 1000, 1500] # vorletztes # 959 letztes 1439

mean_c1 = [7.685214153332941, 8.647939466433439, 8.755318814855267, 8.728635327016125, 8.769999551759987, 8.776702214194092]
std_c1 = [0.10020794506473688, 0.12287807651939232, 0.08248251060547318, 0.08304519492126802, 0.06759433319038019, 0.12659895249777367]

mean_c3 = [7.762886743437322, 8.596458341030935, 8.798446691296359, 8.751792345723775, 8.76805730506273, 8.74081231601401]
std_c3 = [0.11598291894202396, 0.09363631101316408, 0.10748087401514818, 0.09812815045935029, 0.05775935607353134, 0.09677611504981143]

mean_c5 = [7.665914862629701, 8.659614704761324, 8.795426848297684, 8.721246193397805, 8.74803374634143, 8.744422635144518]
std_c5 = [0.10402757843947735, 0.09600572858682611, 0.062457741056709454, 0.07043536838850206, 0.10784576786370587, 0.09917689867200492]

mean_c10 = [7.737014645693776, 8.671485752113487, 8.78130387956802, 8.763744669389167, 8.662650644054375]
std_c10 = [0.09972477960311658, 0.08709899412509303, 0.0974413469033469, 0.0987803915058357, 0.10316599969779615]

"""
plt.plot(steps, mean_is_taul, marker='o', color='green', label='Tau-Leaping')
plt.fill_between(steps, [mean - std for mean, std in zip(mean_is_taul, std_is_taul)],
                 [mean + std for mean, std in zip(mean_is_taul, std_is_taul)],
                 alpha=0.3, color='green')

plt.plot(steps, mean_is_mpl, marker='o', label='MidPoint Tau-Leaping', color='lightcoral')
plt.fill_between(steps, [mean - std for mean, std in zip(mean_is_mpl, std_is_mpl)],
                 [mean + std for mean, std in zip(mean_is_mpl, std_is_mpl)],
                 alpha=0.3)

plt.xlabel('$\\tau$', fontsize=13)
plt.ylabel('Inception Score', fontsize=13)
plt.title('Inception Score of Sampling Methods with Varying $\\tau$', fontsize=14)
plt.legend()
plt.grid(True)
plt.xscale('log')
#plt.yscale('log')
plt.xticks(steps_plot, steps_plot)
plt.savefig('inception_score_tau.pdf')
plt.show()
"""

In [ ]:
# 0: 75:  4.3245049017693304
# 1: 250, 500: [5.845140920974131, 2.252598541276768, 2.3627632134412977]
# 3: 250, 500, [9.18351245256298 2.3423048205916643, [[2.3001590957738642]]
# 5: 75, 250, 500, [12.95, 2.3731947632869037, 2.418666817264665]

# 5: 10, 16, 22, 25, 30, 48, 75, 102
# 3: 10, 14, 20, 32, 50, 75, 102
# 1: 10, 15, 20, 30, 50, 

# 100-1500
import numpy as np
import matplotlib.pyplot as plt
marker_size = 4
line_width = 1.5
"""
pc_1_steps =[5, 10, 20, 30, 50, 100, 818]#, 1000, 1250, 1500]
nfe_01_1_fid = [355.30401756086303, 299.8788404145761, 220.94562240586495, 63.526149427266375, 17.06536557569376, 2.813092226544086, 2.27897546009885]#8, 2.2671159746989906, 2.259, 2.252]
pc_3_steps = [10, 20, 32, 50, 102, 637]#7, 1000, 1250, 1500]
nfe_01_3_fid = [368.2020780292752, 306.44852994200113, 105.99785122999572, 23.8096318610784, 3.8135712497214342, 2.276187915395326]#, 2.2539787186312788, 2.2466063307386435, 2.238]
pc_5_steps =[15, 30, 48, 102, 729]#, 1000, 1250, 1500]
nfe_01_5_fid = [308.4621460802384, 153.52888664861723, 32.976239962089096, 5.501552200792844, 2.2559381898192896]#, 2.2450412804101074, 2.234, 2.2267139529323856]
"""
# 1: 250, 500: [2.252598541276768, 2.3627632134412977]
# 3: [2.3423048205916643, [3.9064362368672505]
pc_1_steps =[15, 20, 30, 50, 75, 100]#, 250]
nfe_01_1_fid = [258.788404145761, 210.94562240586495, 63.526149427266375, 17.06536557569376, 5.845140920974131, 2.813092226544086]#, 2.3627632134412977]
pc_3_steps = [14, 20, 32, 50, 75, 102]#, 250]
nfe_01_3_fid = [328.2020780292752, 278.44852994200113, 105.99785122999572, 23.8096318610784, 9.18351245256298, 3.8135712497214342]#, 2.3423048205916643]
pc_5_steps =[16, 20, 30, 48, 75, 102]#, 250]
nfe_01_5_fid = [348.4621460802384, 320, 153.52888664861723, 32.976239962089096, 12.95, 5.501552200792844]#, 2.418666817264665]

steps = np.array([15, 20, 30, 50, 75, 100])#, 250]) # , 250, 500, 750, 1000, 1500])
fid_taul = np.array([212.90508021598973, 120.04226304249106, 40.6234648618652, 13.453524786963495, 4.3245049017693304, 3.6])#, 2.98404509903506])#, 2.98404509903506, 2.5895556730263706, 2.51, 2.48, 2.460157370950734])


colors = ['black', 'gray', 'lightgray']
colors = ['black', 'gray', (0.7, 0.7, 0.7)]  
colors = ['black', (0.39, 0.39, 0.39), (0.7, 0.7, 0.7)]  # Graustufen-Farben
plt.plot(steps, fid_taul, marker='o', color='green', label='Tau-Leaping', markersize=marker_size, linewidth=line_width)
plt.plot(pc_1_steps, nfe_01_1_fid, marker='o', label='Tau-Leaping Corrector-Steps: 1', color=colors[2], markersize=marker_size, linewidth=line_width)
plt.plot(pc_3_steps, nfe_01_3_fid, marker='o', label='Tau-Leaping Corrector-Steps: 3', color=colors[1], markersize=marker_size, linewidth=line_width)
plt.plot(pc_5_steps, nfe_01_5_fid, marker='o', label='Tau-Leaping Corrector-Steps: 5', color=colors[0], markersize=marker_size, linewidth=line_width)

plt.xlabel('NFE', fontsize=13)
plt.ylabel('FID Value', fontsize=13)
plt.title('FID Value of Tau-Leaping for Different Corrector-Steps', fontsize=14)
plt.legend()
plt.grid(True)

plt.yscale('log')
steps_y = [350, 250, 200, 150, 50, 30, 20, 15, 10, 5, 4, 3]
steps_plot = [15, 20, 30, 50, 75, 100]
plt.yticks(steps_y, steps_y)
plt.xticks(steps_plot, steps_plot)
plt.savefig('fid_nfe_corrector_15100.pdf')
plt.show()

In [ ]:
#plt.plot(nfe_mpl, fid_mpl, marker='o', label='Midpoint Tau-Leaping')
import numpy as np
import matplotlib.pyplot as plt
marker_size = 4
line_width = 1.5

# 0: 75:  4.3245049017693304
# 1: 250, 500: [5.845140920974131, 2.252598541276768, 2.3627632134412977]
# 3: 250, 500, [9.18351245256298 2.3423048205916643, [[2.3001590957738642]]
# 5: 75, 250, 500, [12.95, 2.3731947632869037, 2.418666817264665]

colors = ['black', 'gray', 'lightgray']
colors = ['black', 'gray', (0.7, 0.7, 0.7)]  
colors = ['black', (0.39, 0.39, 0.39), (0.7, 0.7, 0.7)]  # Graustufen-Farben
fid_taul2 = np.array([2.98404509903506, 2.5395556730263706, 2.51, 2.48, 2.47, 2.460157370950734])
nfe_taul2 = [250, 500, 750, 1000, 1250, 1500]

nfe_1_01 = [250, 350, 500, 750, 1000, 1250, 1500] # 1090 last
nfe_1_01_fid = [2.36, 2.3324636154390714,  2.301, 2.288, 2.2711159746989906, 2.259, 2.252]
nfe_3_01 = [250, 350, 500, 750, 1000, 1250, 1500] # vorletzes 956 # 1272
nfe_3_01_fid = [2.392, 2.35941289542929, 2.28, 2.269187915395326, 2.2539787186312788, 2.2466063307386435, 2.238]
nfe_5_01 = [250, 350, 500, 750, 1000, 1250, 1500] #vorletztes 1094 letztes 15000
nfe_5_01_fid = [2.41, 2.3770141451028314, 2.295, 2.2589381898192896, 2.2450412804101074, 2.234, 2.2267139529323856]

plt.plot(nfe_taul2, fid_taul2, marker='o', color='green', label='Tau-Leaping Corrector-Steps: 0', markersize=marker_size, linewidth=line_width)
plt.plot(nfe_1_01, nfe_1_01_fid, marker='o', label='Tau-Leaping Corrector-Steps: 1', color=colors[2], markersize=marker_size, linewidth=line_width)
plt.plot(nfe_3_01, nfe_3_01_fid, marker='o', label='Tau-Leaping Corrector-Steps: 3', color=colors[1], markersize=marker_size, linewidth=line_width)
plt.plot(nfe_5_01, nfe_5_01_fid, marker='o', label='Tau-Leaping Corrector-Steps: 5', color=colors[0], markersize=marker_size, linewidth=line_width)

plt.xlabel('NFE', fontsize=13)
plt.ylabel('FID Value', fontsize=13)
plt.title('FID Value of Tau-Leaping for Different Corrector-Steps', fontsize=14)
plt.legend()
plt.grid(True)

plt.yscale('log')
steps_y = [3, 2.9, 2.8, 2.7, 2.65, 2.6, 2.54, 2.5, 2.4, 2.45, 2.30, 2.26, 2.23, 2.2]
steps_y = [2.4, 2.375, 2.35, 2.325, 2.3, 2.275, 2.25, 2.24, 2.225]
steps_plot = [250, 350, 500, 750, 1000, 1250, 1500]
plt.yticks(steps_y, steps_y)
plt.xticks(steps_plot, steps_plot)
#plt.savefig('fid_nfe_corrector_250500.pdf')
plt.show()

In [ ]:

# state changes TauL: mnist

#np.save('PNGs/state_changes_taul.npy', changes.detach().cpu().numpy())
s_changes_taul = np.load('PNGs/MNIST/state_changes_taul.npy')
#changes[-1, 0, :].reshape(28,28)
shaped_changes = s_changes_taul[:, 0, :].reshape(-1, 28,28)
indices = np.where((shaped_changes[-1, :, :] > 20) & (shaped_changes[-1, :, :] < 250))
print(indices[0])
print(indices[1])
steps = np.linspace(1, 0, 1000)
plt.gca().invert_xaxis()
#for i in range(max_number):
plt.plot(steps, shaped_changes[:, 14, 14])
plt.plot(steps, shaped_changes[:, 9, 8])
plt.plot(steps, shaped_changes[:, 10, 10])
#plt.plot(steps, shaped_changes[:, 13, 7])
plt.plot(steps, shaped_changes[:, 24, 20])
plt.xlabel('Time', fontsize=13)
plt.ylabel('State',fontsize=13)
plt.legend()
plt.title('State Changes during Reverse Process with Tau-Leaping', fontsize=14)
plt.grid(True)
steps_x = [1.0, 0.8, 0.6, 0.4, 0.2, 0]
plt.xticks(steps_x, steps_x)
#plt.savefig('state_changes_taul.pdf')
plt.show()


In [ ]:
# state changes LBJF: mnist

#np.save('PNGs/state_changes_lbjf.npy', changes)
s_changes_lbjf = np.load('PNGs/MNIST/state_changes_lbjf.npy')
#changes[-1, 0, :].reshape(28,28)
shaped_changes = s_changes_lbjf[:, 0, :].reshape(-1, 28,28)
indices = np.where((shaped_changes[-1, :, :] > 20) & (shaped_changes[-1, :, :] < 250))
print(indices[0])
print(indices[1])
steps = np.linspace(1, 0, 1000)
plt.gca().invert_xaxis()
#for i in range(max_number):
plt.plot(steps, shaped_changes[:, 10, 13])
plt.plot(steps, shaped_changes[:, 7, 18])
#plt.plot(steps, shaped_changes[:, 9, 12])
plt.plot(steps, shaped_changes[:, 25, 10])
plt.plot(steps, shaped_changes[:, 17, 14])
plt.xlabel('Time', fontsize=13)
plt.ylabel('State',fontsize=13)
plt.legend()
plt.title('State Changes during Reverse Process with Euler Sampling', fontsize=14)
plt.grid(True)
steps_x = [1.0, 0.8, 0.6, 0.4, 0.2, 0]
plt.xticks(steps_x, steps_x)
#plt.savefig('state_changes_lbjf.pdf')
plt.show()

In [ ]:
# multiple jumps in a single dim: MNIST

#np.save('PNGs/jumps_over_1_taul.npy', changes)
changes_jump_over1 = np.load('PNGs/MNIST/jumps_over_1_taul.npy')
steps = np.linspace(1, 0, len(changes_jump_over1[:-1]))

plt.gca().invert_xaxis()
plt.plot(steps, changes_jump_over1[:-1], color='black')
plt.xlabel('Time', fontsize=13)
plt.ylabel('Avg. Proportion of Multiple Jumps', fontsize=13)
plt.title('Avg. Proportion of Multiple Jumps in one Dim. for Tau-Leaping', fontsize=14)
plt.legend()
plt.grid(True)
#plt.savefig('jumps_over_1_taul.pdf')
plt.show()


In [ ]:
#np.save('PNGs/changing_dim_lbjf.npy', changes)
#np.save('PNGs/changing_dim_taul.npy', changes_jump)
changes_dim = np.load('changing_dims_mnist_taul.npy')
changes_jump_taul = np.load('PNGs/MNIST/changing_dim_taul.npy')
changes_jump_lbjf = np.load('PNGs/MNIST/changing_dim_lbjf.npy')
steps = np.linspace(1, 0, len(changes_jump_taul[:-1]))
plt.gca().invert_xaxis()
plt.plot(steps, changes_jump_lbjf[:-1], label='Euler')
plt.plot(steps, changes_dim[:-1], label='Tau-Leaping', color='green')
#plt.plot(steps, changes_dim[:-1], label='Tau-Leaping 2')
plt.xlabel('Time', fontsize=13)
plt.ylabel('Avg. Proportion of Changing Dimensions', fontsize=13)
plt.legend()
plt.grid(True)
steps_y = [1, 0.8, 0.6, 0.4, 0.2, 0.1]
plt.yticks(steps_y, steps_y)
#plt.savefig('changing_dim.pdf')
plt.show()